##1. 기간을 지정해서 서브레딧 크롤링 하는 방법

In [1]:
import math
import json
import requests
import itertools
import numpy as np
import praw
import pandas as pd
from psaw import PushshiftAPI    #library Pushshift
import datetime as dt            #library for date management
import p                         #library for data manipulation
import matplotlib.pyplot as plt  #library for plotting


reddit = praw.Reddit(client_id="your client id",                        # your client id
                                client_secret="your client secret",     # your client secret
                                user_agent="your user agent",           # your user agent
                                username="your reddit username",        # your reddit username
                                password="Password")                    # your reddit password

subreddit = reddit.subreddit("redditdev")

Version 7.5.0 of praw is outdated. Version 7.6.0 was released Tuesday May 10, 2022.


In [2]:
api = PushshiftAPI()  

In [8]:
def data_prep_posts(subreddit, start_time, end_time, filters):#, limit):
    if(len(filters) == 0):
        filters = ['id', 'author', 'created_utc',
                   'domain', 'url',
                   'title', 'selftext','num_comments']                 
                   #We set by default some useful columns

    posts = list(api.search_submissions(
        subreddit=subreddit,   #Subreddit we want to audit
        after=start_time,      #Start date
        before=end_time,       #End date
        filter=filters))#,        #Column names we want to retrieve
        #limit=limit))          ##Max number of posts

    return pd.DataFrame(posts) #Return dataframe for analysis


In [105]:
def main():
    subreddit = "cooking"                           #Name of the subreddit we are auditing
    start_time = int(dt.datetime(2022, 1, 1).timestamp())  #We define the starting date for our search
    end_time = int(dt.datetime(2022, 3, 30).timestamp())   #We define the ending date for our search
    filters = []                                           #We don´t want specific filters
    #limit = 1000                                           #Number of elelemts we want to recieve

    """Here we are going to get subreddits for a brief analysis"""
    df_p = data_prep_posts(subreddit,start_time,
                         end_time,filters)#,limit)           #Call function for dataframe creation of comments

    df_p['datetime'] = df_p['created_utc'].map(
        lambda t: dt.datetime.fromtimestamp(t))
    df_p = df_p.drop('created_utc', axis=1)                #Drop the column on timestamp format
    df_p = df_p.sort_values(by='datetime')                 #Sort the Row by datetime
    df_p["datetime"] = pd.to_datetime(df_p["datetime"])    #Convert timestamp format to datetime for data analysis


    df_p.to_csv(f'dataset_{subreddit}_posts5.csv', sep=',', # Save the dataset on a csv file for future analysis
                header=True, index=False, columns=[
            'id', 'author', 'datetime', 'domain',
            'url', 'title', 'selftext', 'num_comments'
        ])

In [106]:
if __name__== "__main__" : main()

##2. 데이터 불러와서 데이터 프레임으로 다듬기 

In [107]:
data = pd.read_csv("dataset_cooking_posts5.csv")
data

,id,author,datetime,domain,url,title,selftext,num_comments
0,rswe8m,HurtlinTurtlin,2022-01-01 00:34:29,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswe...,Low on skill but rich with time and want to ma...,Howdy! I am a baker who knows the basics of co...,0
1,rswg5a,snowsunshyne,2022-01-01 00:37:02,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswg...,Pots and pans stains,Hi. I just got cuisinart multiclad pots and pa...,0
2,rswmbc,kanaora,2022-01-01 00:45:19,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswm...,Average Scoville Tolerance,Is there an average Scoville tolerance that mo...,0
3,rswpqk,MadMan1784,2022-01-01 00:49:57,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswp...,Any last-minute gluten free NYE potluck ideas?,So my cousin is allergic to gluten (the only o...,0
4,rswtow,liquor_ibrlyknoher,2022-01-01 00:55:17,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswt...,Black garlic,"I'm interested in making this, the recipe is s...",0
...,...,...,...,...,...,...,...,...
12393,tr0rl9,Soupbarn,2022-03-29 23:17:12,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr0r...,Pairing Flavors,I have a very simple recipe for roasting red o...,0
12394,tr1m30,snap-NatalieCruz0020,2022-03-29 23:37:06,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr1m...,hahahalol add me on my snapchat nataliecruz0020,[removed],6
12395,tr1p0m,snap-NatalieCruz0020,2022-03-29 23:41:02,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr1p...,add me on my snapchat let's have some fun. nat...,[removed],2
12396,tr2a8z,LectureQuirky3234,2022-03-29 23:56:33,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr2a...,What is your favourite 'fusion cooking' ingred...,I like to use japanese yam as a thickener and ...,7


In [108]:
data = data.dropna()

In [109]:
#selftext가 지워져있는 경우 해당 열 삭제하기
mask = data['selftext'].isin(['[removed]'])
data1 = data[~mask]
data1

,id,author,datetime,domain,url,title,selftext,num_comments
0,rswe8m,HurtlinTurtlin,2022-01-01 00:34:29,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswe...,Low on skill but rich with time and want to ma...,Howdy! I am a baker who knows the basics of co...,0
1,rswg5a,snowsunshyne,2022-01-01 00:37:02,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswg...,Pots and pans stains,Hi. I just got cuisinart multiclad pots and pa...,0
2,rswmbc,kanaora,2022-01-01 00:45:19,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswm...,Average Scoville Tolerance,Is there an average Scoville tolerance that mo...,0
3,rswpqk,MadMan1784,2022-01-01 00:49:57,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswp...,Any last-minute gluten free NYE potluck ideas?,So my cousin is allergic to gluten (the only o...,0
4,rswtow,liquor_ibrlyknoher,2022-01-01 00:55:17,self.Cooking,https://www.reddit.com/r/Cooking/comments/rswt...,Black garlic,"I'm interested in making this, the recipe is s...",0
...,...,...,...,...,...,...,...,...
12389,tqzmmh,zooeva,2022-03-29 22:22:04,self.Cooking,https://www.reddit.com/r/Cooking/comments/tqzm...,Advice for choose between two stainless steel ...,I have done some research regarding good quali...,0
12390,tr03gi,Final_Cress_9734,2022-03-29 22:45:01,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr03...,Why do people try to make new dishes out of th...,I just saw a post about someone trying to figu...,0
12391,tr03m2,Inside-Strawberry517,2022-03-29 22:45:12,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr03...,Need some recipe help for 2 deer camp soups.,I grew up hunting with my grandpa in northwest...,0
12393,tr0rl9,Soupbarn,2022-03-29 23:17:12,self.Cooking,https://www.reddit.com/r/Cooking/comments/tr0r...,Pairing Flavors,I have a very simple recipe for roasting red o...,0


In [110]:
#필요 없는 열 정리하고 새로운 칼럼 추가
data1 = data1.drop(['author', 'datetime','domain','url','num_comments'], axis=1)
data1.insert(3, 'label', 0)

In [111]:
data1.head()

,id,title,selftext,label
0,rswe8m,Low on skill but rich with time and want to ma...,Howdy! I am a baker who knows the basics of co...,0
1,rswg5a,Pots and pans stains,Hi. I just got cuisinart multiclad pots and pa...,0
2,rswmbc,Average Scoville Tolerance,Is there an average Scoville tolerance that mo...,0
3,rswpqk,Any last-minute gluten free NYE potluck ideas?,So my cousin is allergic to gluten (the only o...,0
4,rswtow,Black garlic,"I'm interested in making this, the recipe is s...",0


In [112]:
#data1.reset_index(drop=True, inplace=True) #인덱스정리
data1.to_csv("train_cooking_new.csv")
data1

,id,title,selftext,label
0,rswe8m,Low on skill but rich with time and want to ma...,Howdy! I am a baker who knows the basics of co...,0
1,rswg5a,Pots and pans stains,Hi. I just got cuisinart multiclad pots and pa...,0
2,rswmbc,Average Scoville Tolerance,Is there an average Scoville tolerance that mo...,0
3,rswpqk,Any last-minute gluten free NYE potluck ideas?,So my cousin is allergic to gluten (the only o...,0
4,rswtow,Black garlic,"I'm interested in making this, the recipe is s...",0
...,...,...,...,...
12389,tqzmmh,Advice for choose between two stainless steel ...,I have done some research regarding good quali...,0
12390,tr03gi,Why do people try to make new dishes out of th...,I just saw a post about someone trying to figu...,0
12391,tr03m2,Need some recipe help for 2 deer camp soups.,I grew up hunting with my grandpa in northwest...,0
12393,tr0rl9,Pairing Flavors,I have a very simple recipe for roasting red o...,0
